# Tratamento dos dados

In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Carregando os arquivos e salvando em um dataframe
file = "dados_compras.json"
compras = pd.read_json(file, orient = "records")
compras.head()

,Login,Idade,Sexo,Item ID,Nome do Item,Valor
0,Aelalis34,38,Masculino,165,Bone Crushing Silver Skewer,3.37
1,Eolo46,21,Masculino,119,"Stormbringer, Dark Blade of Ending Misery",2.32
2,Assastnya25,34,Masculino,174,Primitive Blade,2.46
3,Pheusrical25,21,Masculino,92,Final Critic,1.36
4,Aela59,23,Masculino,63,Stormfury Mace,1.27


In [ ]:
#verificar estatísticas das colunas do dataframe (para ser utilizado mais tarde para entedner se os futuros dados obitos fazem sentido)
print(compras.describe())

# Informações dos Consumidores
* Número total de consumidores
* Número total de vendas

In [ ]:
#verificar o total de vendas e de consumidores (se há mais de uma venda por consumidor)
demografico = compras.loc[:, ["Login", "Sexo","Idade"]]
demografico.head()

In [ ]:
players = demografico.drop_duplicates()
players

# Análise Geral de Compras
* Número de itens exclusivos
* Preço médio de compra
* Número total de compras
* Rendimento total

In [43]:
media_vendas = compras['Valor'].mean()
total_vendas = compras['Valor'].sum()
numero_vendas = compras['Valor'].count()
item_unicos = len(compras["Item ID"].unique())#contagem de str

vendas = pd.DataFrame({"Media de Vendas": [media_vendas],# valores escalares entre []
                      "Total de Vendas": total_vendas,  
                      "Numero de Vendas": numero_vendas,
                      "Itens Únicos": item_unicos})

vendas["Media de Vendas"] = vendas["Media de Vendas"].map("${:,.2f}".format)
vendas["Total de Vendas"] = vendas["Total de Vendas"].map("${:,.2f}".format)
vendas["Numero de Vendas"] = vendas["Numero de Vendas"].map("{:}".format)
vendas["Itens Únicos"] = vendas["Itens Únicos"].map("{:}".format)
vendas


,Media de Vendas,Total de Vendas,Numero de Vendas,Itens Únicos
0,$2.93,"$2,286.33",780,183


# Informações Demográficas Por Gênero

* Porcentagem e contagem de compradores masculinos
* Porcentagem e contagem de compradores do sexo feminino
* Porcentagem e contagem de outros / não divulgados

In [42]:
masc = len(compras.loc[compras['Sexo'] == 'Masculino'])
fem = len(compras.loc[compras["Sexo"] == 'Feminino'])
others = len(compras.loc[compras['Sexo'] == 'Outro / Não Divulgado'])
total_sex = compras['Sexo'].count()
masc_por = (masc / total_sex)*100
fem_por = (fem / total_sex)*100
others_por = (others / total_sex)*100
generos = pd.DataFrame({"Quantidade": [masc, fem, others],
                        "%": [masc_por,fem_por, others_por]}, 
                        index= ["Masculino", "Feminino", "Outros / Não Divulgado"])
generos['%'] = generos['%'].map("{:.2f}%".format)
generos

,Quantidade,%
Masculino,633,81.15%
Feminino,136,17.44%
Outros / Não Divulgado,11,1.41%


# Análise de Compras Por Gênero 

* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Compras for faixa etária

In [9]:
total = compras.groupby(['Sexo']).sum()['Valor'].map("$ {:,.2f}".format)
media = compras.groupby(['Sexo']).mean()['Valor'].map("$ {:.2f}".format)
qntd = compras.groupby(['Sexo']).count()['Valor']
genero = pd.DataFrame({"Total de Compras": total,
                        "Preço Médio": media,
                        "Quantidade de Compras": qntd})
genero


,Total de Compras,Preço Médio,Quantidade de Compras
Sexo,,,
Feminino,$ 382.91,$ 2.82,136
Masculino,"$ 1,867.68",$ 2.95,633
Outro / Não Divulgado,$ 35.74,$ 3.25,11


In [ ]:
#faixa etária ##lemberet tentar groupyby com listas das idades
total = compras.groupby(['Sexo']).sum()['Valor'].map("$ {:,.2f}".format)
media = compras.groupby(['Sexo']).mean()['Valor'].map("$ {:.2f}".format)
qntd = compras.groupby(['Sexo']).count()['Valor']
genero = pd.DataFrame({"Total de Compras": total,
                        "Preço Médio": media,
                        "Quantidade de Compras": qntd})
genero


# Identifique os 5 principais compradores pelo valor total de compra e, em seguida, liste (em uma tabela):

* Login
* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Itens mais populares

In [ ]:
total_compras = compras.groupby(["Login"]).sum()["Valor"].map("${:.2f}".format)
media_compras = compras.groupby(["Login"]).mean()["Valor"].map("${:.2f}".format) 
qntd_compras = compras.groupby(["Login"]).count()["Valor"]

tabela = pd.DataFrame({"Total Compras": total_compras,
                       "Numero Compras": qntd_compras,
                       "Media Compras": media_compras,})
tabela.sort_values("Total Compras", ascending=False).head(5)##inserir nome da coluna a ser ordenada

#  Identifique os 5 itens mais populares por contagem de compras e, em seguida, liste (em uma tabela):

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra
* Itens mais lucrativos

In [26]:
items = compras['Nome do Item'].drop_duplicates().unique()
items

array(['Bone Crushing Silver Skewer',
       'Stormbringer, Dark Blade of Ending Misery', 'Primitive Blade',
       'Final Critic', 'Stormfury Mace', 'Sleepwalker', 'Mercenary Sabre',
       'Interrogator, Blood Blade of the Queen',
       'Ghost Reaver, Longsword of Magic',
       'Expiration, Warscythe Of Lost Worlds',
       'Despair, Favor of Due Diligence', 'Alpha, Reach of Ending Hope',
       'Dreamkiss', 'Piety, Guardian of Riddles', 'Bonecarvin Battle Axe',
       'Blood-Forged Skeletal Spine', "Twilight's Carver",
       'Lightning, Etcher of the King', 'Celeste',
       'Winterthorn, Defender of Shifting Worlds',
       'Glimmer, Ender of the Moon', 'Phantomlight', 'Brimstone',
       "Dragon's Greatsword", 'Conqueror Adamantite Mace', 'Persuasion',
       'Crying Steel Sickle', 'The Oculus, Token of Lost Worlds',
       'Glinting Glass Edge', 'War-Forged Gold Deflector',
       'Shadow Strike, Glory of Ending Hope',
       'Rage, Legacy of the Lone Victor', 'Betrayer', 'Ser

In [ ]:
# Cálculos básicos
item_total = items.groupby(["Nome do Item"]).sum()["Valor"]
item_medio = items.groupby(["Nome do Item"]).mean()["Valor"]
item_qntd = items.groupby(["Nome do Item"]).count()["Valor"]

# Dataframe para os resultados
table = pd.DataFrame({"Valor Total de Compra": [item_total],
                          "Valor Médio de Compra": [item_medio],
                          "Número de Compras": [item_qntd]})

table.sort_values("Valor Total de Compra", ascending=True).head(5)

# Identifique os 5 itens mais lucrativos pelo valor total de compra e, em seguida, liste (em uma tabela):

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra